**Load Librairies**

In [1]:
import asyncio
import gc
import os
import re
from typing import NoReturn

import librosa
import numpy as np
import pandas as pd
import pylab
import yt_dlp as yt

**Load Data To Download**

In [2]:
df = pd.read_csv("../../data/scraped/complete_dataset.csv", sep=",")[27170:]

**Download Audio Files & Automatic Conversion To CSV**

In [3]:
class YTDLSource:
    @classmethod
    async def from_url(cls, url, ytdl, *, loop=None):
        loop = loop or asyncio.get_event_loop()
        data = await loop.run_in_executor(
            None, lambda: ytdl.extract_info(url)
        )
        if "entries" in data:
            data = data["entries"][0]
        return ytdl.prepare_filename(data)

In [4]:
def save_spectrogram_image(spectrogram, output_file, y_axis):
    pylab.figure(figsize=(12, 8))
    pylab.axis('off')  # no axis
    pylab.axes([0., 0., 1., 1.], frameon=False, xticks=[], yticks=[])  # Remove the white edge
    librosa.display.specshow(spectrogram, y_axis=y_axis, x_axis='time', fmax=None, cmap='viridis')
    pylab.savefig(output_file, dpi=300, bbox_inches=None, pad_inches=0)
    pylab.close()

In [5]:
async def download_audio_files(row, loop) -> NoReturn:
    try:
        filename = f"{row[0]}_{row[1]}".replace(" ", "_")
        filename = re.sub('[^\w\s]', '', filename)
        path = f"../../data/musics/{filename}"
        audio_file = f"{path}.wav"
        ytdl_format_options = {
            "outtmpl": path,
            "format": "bestaudio/best",
            "postprocessors": [
                {
                    "key": "FFmpegExtractAudio",
                    "preferredcodec": "wav",
                    "preferredquality": "192",
                }
            ],
            "noplaylist": False,
            "nocheckcertificate": True,
            "ignoreerrors": False,
            "quiet": True,
            "no_warnings": True,
            "default_search": "auto",
        }
        ytdl = yt.YoutubeDL(ytdl_format_options)
        await asyncio.wait_for(YTDLSource.from_url(f"{row[0]}_{row[1]}", ytdl=ytdl, loop=loop), timeout=60)

        y, sr = librosa.load(audio_file)

        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        np.savetxt(f"../../data/dataset/raw/mfcc/{row[2]}/{filename}.csv", mfcc, delimiter=",")
        save_spectrogram_image(mfcc, f"../../data/dataset/img/mfcc/{row[2]}/{filename}.png", y_axis='linear')
        print(f"MFCCed : {audio_file}")

        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        np.savetxt(f"../../data/dataset/raw/chroma/{row[2]}/{filename}.csv", chroma_stft, delimiter=",")
        save_spectrogram_image(chroma_stft, f"../../data/dataset/img/chroma/{row[2]}/{filename}.png",
                               y_axis='chroma')
        print(f"Chromaed : {audio_file}")

        os.remove(audio_file)
        del y, sr, mfcc, chroma_stft
    except Exception:
        print(f"Error : {audio_file}")

In [6]:
async def start():
    for row in df.values:
        await download_audio_files(row, asyncio.get_event_loop())
        gc.collect()

In [ ]:
await start()

              MFCCed : ../../data/musics/fiona_apple_sullen_girl.wav
Chromaed : ../../data/musics/fiona_apple_sullen_girl.wav
              MFCCed : ../../data/musics/fiona_apple_pale_september.wav
Chromaed : ../../data/musics/fiona_apple_pale_september.wav
              MFCCed : ../../data/musics/fiona_apple_get_gone.wav
Chromaed : ../../data/musics/fiona_apple_get_gone.wav
              MFCCed : ../../data/musics/fiona_apple_i_know.wav
Chromaed : ../../data/musics/fiona_apple_i_know.wav
              MFCCed : ../../data/musics/fiona_apple_across_the_universe.wav
Chromaed : ../../data/musics/fiona_apple_across_the_universe.wav
              MFCCed : ../../data/musics/firehouse_love_of_a_lifetime.wav
Chromaed : ../../data/musics/firehouse_love_of_a_lifetime.wav
              MFCCed : ../../data/musics/firehouse_reach_for_the_sky.wav
Chromaed : ../../data/musics/firehouse_reach_for_the_sky.wav
              MFCCed : ../../data/musics/firehouse_when_i_look_into_your_eyes.wav
Chromaed : .

[download] Got error: HTTP Error 403: Forbidden


Error : ../../data/musics/gravity_kills_guilty.wav
              MFCCed : ../../data/musics/great_big_sea_ordinary_day.wav
Chromaed : ../../data/musics/great_big_sea_ordinary_day.wav
              MFCCed : ../../data/musics/greater_vision_my_name_is_lazarus.wav
Chromaed : ../../data/musics/greater_vision_my_name_is_lazarus.wav
              MFCCed : ../../data/musics/green_day_scattered.wav
Chromaed : ../../data/musics/green_day_scattered.wav
              MFCCed : ../../data/musics/green_day_redundant.wav
Chromaed : ../../data/musics/green_day_redundant.wav
              MFCCed : ../../data/musics/green_day_the_grouch.wav
Chromaed : ../../data/musics/green_day_the_grouch.wav
              MFCCed : ../../data/musics/green_day_good_riddance_time_of_your_life.wav
Chromaed : ../../data/musics/green_day_good_riddance_time_of_your_life.wav
              MFCCed : ../../data/musics/green_day_uptight.wav
Chromaed : ../../data/musics/green_day_uptight.wav
              MFCCed : ../../data/music

[download] Got error: HTTP Error 403: Forbidden


Error : ../../data/musics/jeff_buckley_mojo_pin.wav
              MFCCed : ../../data/musics/jellyfish_new_mistake.wav
Chromaed : ../../data/musics/jellyfish_new_mistake.wav
              MFCCed : ../../data/musics/jellyfish_joining_a_fan_club.wav
Chromaed : ../../data/musics/jellyfish_joining_a_fan_club.wav
              MFCCed : ../../data/musics/jellyfish_that_is_why.wav
Chromaed : ../../data/musics/jellyfish_that_is_why.wav
              MFCCed : ../../data/musics/jennifer_lopez_waiting_for_tonight.wav
Chromaed : ../../data/musics/jennifer_lopez_waiting_for_tonight.wav
              MFCCed : ../../data/musics/jennifer_love_hewitt_cool_with_you.wav
Chromaed : ../../data/musics/jennifer_love_hewitt_cool_with_you.wav
              MFCCed : ../../data/musics/jermaine_dupri_welcome_to_atlanta.wav
Chromaed : ../../data/musics/jermaine_dupri_welcome_to_atlanta.wav
              MFCCed : ../../data/musics/jerry_butler_moody_woman.wav
Chromaed : ../../data/musics/jerry_butler_moody_woman.wa

...

Error : ../../data/musics/the_cranberries_i_cant_be_with_you.wav
              MFCCed : ../../data/musics/the_cranberries_ridiculous_thoughts.wav
Chromaed : ../../data/musics/the_cranberries_ridiculous_thoughts.wav
              MFCCed : ../../data/musics/the_cranberries_the_icicle_melts.wav
Chromaed : ../../data/musics/the_cranberries_the_icicle_melts.wav
              MFCCed : ../../data/musics/the_cranberries_everything_i_said.wav
Chromaed : ../../data/musics/the_cranberries_everything_i_said.wav
              MFCCed : ../../data/musics/the_cure_apart.wav
Chromaed : ../../data/musics/the_cure_apart.wav
              MFCCed : ../../data/musics/the_cure_to_wish_impossible_things.wav
Chromaed : ../../data/musics/the_cure_to_wish_impossible_things.wav
              MFCCed : ../../data/musics/the_cure_from_the_edge_of_the_deep_green_sea.wav
Chromaed : ../../data/musics/the_cure_from_the_edge_of_the_deep_green_sea.wav
              MFCCed : ../../data/musics/the_cure_a_letter_to_elise.wav

[download] Got error: HTTP Error 403: Forbidden


Error : ../../data/musics/the_gories_nitroglycerine.wav
              MFCCed : ../../data/musics/the_gories_hey_hey_were_the_gories.wav
Chromaed : ../../data/musics/the_gories_hey_hey_were_the_gories.wav
              MFCCed : ../../data/musics/the_gourds_gangsta_lean.wav
Chromaed : ../../data/musics/the_gourds_gangsta_lean.wav
              MFCCed : ../../data/musics/the_grace_thrillers_make_us_one.wav
Chromaed : ../../data/musics/the_grace_thrillers_make_us_one.wav
              MFCCed : ../../data/musics/the_greenhornes_the_end_of_the_night.wav
Chromaed : ../../data/musics/the_greenhornes_the_end_of_the_night.wav
              MFCCed : ../../data/musics/the_guess_who_follow_your_daughter_home.wav
Chromaed : ../../data/musics/the_guess_who_follow_your_daughter_home.wav
              MFCCed : ../../data/musics/the_herbaliser_a_mother.wav
Chromaed : ../../data/musics/the_herbaliser_a_mother.wav
              MFCCed : ../../data/musics/the_highwaymen_here_comes_that_rainbow_again.wav
Ch

[download] Got error: HTTP Error 403: Forbidden


Error : ../../data/musics/the_smashing_pumpkins_geek_usa.wav
              MFCCed : ../../data/musics/the_smashing_pumpkins_quiet.wav
Chromaed : ../../data/musics/the_smashing_pumpkins_quiet.wav
              MFCCed : ../../data/musics/the_smashing_pumpkins_suffer.wav
Chromaed : ../../data/musics/the_smashing_pumpkins_suffer.wav
              MFCCed : ../../data/musics/the_smashing_pumpkins_window_paine.wav
Chromaed : ../../data/musics/the_smashing_pumpkins_window_paine.wav
              MFCCed : ../../data/musics/the_smashing_pumpkins_bury_me.wav
Chromaed : ../../data/musics/the_smashing_pumpkins_bury_me.wav
              MFCCed : ../../data/musics/the_smashing_pumpkins_daydream.wav
Chromaed : ../../data/musics/the_smashing_pumpkins_daydream.wav
              MFCCed : ../../data/musics/the_smashing_pumpkins_tristessa.wav
Chromaed : ../../data/musics/the_smashing_pumpkins_tristessa.wav
              MFCCed : ../../data/musics/the_smashing_pumpkins_snail.wav
Chromaed : ../../data/musics